In [11]:
import sys
import os
analysis_dir_path = '../'
sys.path.append(analysis_dir_path)

import models
import pandas as pd

In [12]:
filters = 32
kernels = 5
units = 128

In [13]:
CNN = f'CNN_{filters}_filters_{kernels}_kernels_predictions'
CNNA = f'CNN_Attention_{filters}_filters_{kernels}_kernels_predictions'
RNN = f'RNN_{units}_units_predictions'
LSTM = f'LSTM_{units}_layers_predictions'

In [14]:
MODEL = LSTM
COMMODITY = 'nickel_shift_new_20'
RULE_NUM = 20
THRESHOLD = 0.35
EXCLUDE = ['CNNA']
confidence_levels = [0.4, 0.45, 0.5, 0.55, 0.6, 0.7, 0.8, 0.9, 0.95]

algo = ['correction', 'detection_correction']
ALGO = algo[1]

model_name = 'CNN_256_filters_5_kernels_predictions'
df_path = f'../{COMMODITY}/test/predictions/test/{model_name}.csv'


In [15]:
# df = models.npy_to_bowpy(f'{COMMODITY}/test/predictions/test/{MODEL}_predictions.csv', f'copper/test/predictions/test_{METRIC}_{ALGO}/', confidence_levels)
# df = models.npy_to_top_n_f1_bowpy(f'../{COMMODITY}/test/predictions/test/{MODEL}.csv', f'../{COMMODITY}/test/predictions/test_F1_{ALGO}', RULE_NUM)
# df = models.npy_to_threshold_f1_bowpy(f'../{COMMODITY}/test/predictions/test/{MODEL}.csv', f'../{COMMODITY}/test/predictions/test_F1_{ALGO}', THRESHOLD, EXCLUDE)
df = models.npy_to_top_n_f1_bowpy(df_path, f'../{COMMODITY}/test/predictions/test_{ALGO}', RULE_NUM, None)
df

c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

Excluded 0 rules due to model filtering


c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\manim\LabV2\ONR-Price-Prediction\onr_price_prediction\venv\Lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to cont

,pred,corr,true_positive,false_positive,rule0,rule1,rule2,rule3,rule4,rule5,...,rule10,rule11,rule12,rule13,rule14,rule15,rule16,rule17,rule18,rule19
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
426,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
427,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
428,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [16]:
def calculate_metrics(df, rule_result_column):
    TP = ((df[rule_result_column] == 1) & (df['corr'] == 1)).sum()
    FP = ((df[rule_result_column] == 1) & (df['corr'] == 0)).sum()
    FN = ((df[rule_result_column] == 0) & (df['corr'] == 1)).sum()
    
    precision = TP / (TP + FP) if (TP + FP) > 0 else 0
    recall = TP / (TP + FN) if (TP + FN) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0
    
    return f1, recall, precision

In [17]:
f1, recall, precision = calculate_metrics(df, "pred")


In [18]:
df.to_numpy().dump('data/test.npy')

In [19]:
import example
df = example.run_edcr()
df

quantity:0.305
NCi:[]
cci:[14, 7, 8, 9], new_pre:0.5312416993484477, pre:0.4027777777777778
TP:37, FP:63, TN:306, FN:24
ep:0.005
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
quantity:0.36600000000000005
NCi:[]
cci:[14, 7, 8, 9], new_pre:0.5312416993484477, pre:0.4027777777777778
TP:37, FP:63, TN:306, FN:24
ep:0.006
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
quantity:0.42700000000000005
NCi:[]
cci:[14, 7, 8, 9], new_pre:0.5312416993484477, pre:0.4027777777777778
TP:37, FP:63, TN:306, FN:24
ep:0.007
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
quantity:0.4880000000000001
NCi:[]
cci:[14, 7, 8, 9], new_pre:0.5312416993484477, pre:0.4027777777777778
TP:37, FP:63, TN:306, FN:24
ep:0.008
[0.37, 0.6065573770491803, 0.45962732919254656, 0, 28, 0, 4]
quantity:0.5490000000000002
NCi:[]
cci:[14, 7, 8, 9], new_pre:0.5312416993484477, pre:0.4027777777777778
TP:37, FP:63, TN:306, FN:24
ep:0.009000000000000001
[0.37, 0.6065573770491803, 0.4596273291925

,epsilon,pre,recall,F1,NSC,PSC,NRC,PRC
0,0.005,0.37,0.606557,0.459627,0,28,0,4
1,0.006,0.37,0.606557,0.459627,0,28,0,4
2,0.007,0.37,0.606557,0.459627,0,28,0,4
3,0.008,0.37,0.606557,0.459627,0,28,0,4
4,0.009,0.37,0.606557,0.459627,0,28,0,4
...,...,...,...,...,...,...,...,...
290,0.295,0.37,0.606557,0.459627,0,28,0,4
291,0.296,0.37,0.606557,0.459627,0,28,0,4
292,0.297,0.37,0.606557,0.459627,0,28,0,4
293,0.298,0.37,0.606557,0.459627,0,28,0,4


In [20]:
print(f'Base model results\nf1: {f1}\nrecall: {recall}\nprecision: {precision}')

Base model results
f1: 0.43609022556390975
recall: 0.47540983606557374
precision: 0.4027777777777778


In [21]:
def model_select(csv_filepath):
    df = pd.read_csv(csv_filepath, nrows=32)

    # Remove rows where any of the precision or recall for 0 or 1 is 0 or 1
    filtered_df = df[(df[['Precision (0)', 'Recall (0)', 'Precision (1)', 'Recall (1)']] != 0).all(axis=1) &
                     (df[['Precision (0)', 'Recall (0)', 'Precision (1)', 'Recall (1)']] != 1).all(axis=1)]

    if filtered_df.empty:
        print("No models meet the criteria.")
        return []

    # Filter by precision-recall difference
    filtered_df = filtered_df[(filtered_df['Precision (1)'] - filtered_df['Recall (1)']).abs() <= 0.6]
    
    # Initialize a dictionary to store model data and a set to track used models
    models = {}
    used_models = set()

    # Define metric priorities and criteria
    metric_criteria = [
        ('F1 (1)', False, 'Highest F1'),
        ('Recall (1)', False, 'Highest Recall'),
        ('Precision (1)', False, 'Highest Precision'),
        ('F1 (1)', True, 'Lowest F1')
    ]

    for metric, ascending, label in metric_criteria:
        # Sort and filter for the best or worst model based on the metric criteria
        best_model = filtered_df.sort_values(by=metric, ascending=ascending).drop(index=used_models).iloc[0] if not filtered_df.empty else None
        
        if best_model is not None:
            model_id = best_model.name  # Assuming the index is the model ID
            models[label] = {
                'model': format_model_name(best_model),
                'metrics': best_model[['Precision (1)', 'Recall (1)', 'F1 (1)']].to_dict()
            }
            used_models.add(model_id)
        else:
            print(f"No suitable model found for {label} after precision-recall filtering.")

    # Print the selected metric values for verification
    for key, value in models.items():
        print(f"{key}: {value['metrics']} at model - {value['model']}")

    return models

In [22]:
import example as ep
models = ep.model_select(f'../cobalt_shift_new_20/test/results_test.csv')
print(models)
# print(COMMODITY)

Highest F1: {'Precision (1)': 0.8225806451612904, 'Recall (1)': 0.6538461538461539, 'F1 (1)': 0.7285714285714286} at model - CNN_256_filters_3_kernels_predictions
Highest Recall: {'Precision (1)': 0.8225806451612904, 'Recall (1)': 0.6538461538461539, 'F1 (1)': 0.7285714285714286} at model - CNN_256_filters_3_kernels_predictions
Highest Precision: {'Precision (1)': 0.9375, 'Recall (1)': 0.1923076923076923, 'F1 (1)': 0.3191489361702128} at model - CNN_Attention_128_filters_5_kernels_predictions
Lowest F1: {'Precision (1)': 0.25, 'Recall (1)': 0.0128205128205128, 'F1 (1)': 0.024390243902439} at model - LSTM_32_layers_predictions
{'Highest F1': {'model': 'CNN_256_filters_3_kernels_predictions', 'metrics': {'Precision (1)': 0.8225806451612904, 'Recall (1)': 0.6538461538461539, 'F1 (1)': 0.7285714285714286}}, 'Highest Recall': {'model': 'CNN_256_filters_3_kernels_predictions', 'metrics': {'Precision (1)': 0.8225806451612904, 'Recall (1)': 0.6538461538461539, 'F1 (1)': 0.7285714285714286}}, '